In [1]:
import pandas as pd

In [13]:
from rbbm_src.labelling_func_src.src.example_tree_rules import *
from rbbm_src.labelling_func_src.src.LFRepair import run_snorkel
from rbbm_src.labelling_func_src.src.classes import clean_text
import psycopg2
import pandas as pd
from snorkel.labeling import (
	LabelingFunction, 
	labeling_function, 
	PandasLFApplier, 
	LFAnalysis,
	filter_unlabeled_dataframe
	)
from snorkel.labeling.model import MajorityLabelVoter, LabelModel
from sentence_transformers import SentenceTransformer, util
import numpy as np


In [6]:
conn = psycopg2.connect(dbname='label', user='postgres')

In [7]:
amazon_rules = gen_amazon_funcs()
lfs=[f.gen_label_rule() for f in amazon_rules]


In [8]:
# logger.critical(LFs)
sentences_df=pd.read_sql(f'SELECT * FROM amazon01', conn)
sentences_df = sentences_df.rename(columns={"class": "expected_label", "content": "old_text"})
sentences_df['text'] = sentences_df['old_text'].apply(lambda s: clean_text(s))
sentences_df = sentences_df[~sentences_df['text'].isna()]

# # lf_internal_args = lf_input_internal(funcs=LFs)

# # # Snorkel built-in labelling function applier
# applier = PandasLFApplier(lfs=lfs)

# # # Apply the labelling functions to get vectors
# initial_vectors = applier.apply(df=sentences_df, progress_bar=False)
# # # logger.critical(initial_vectors)
# # # df.to_csv('snorkel.csv')

# # if(lf_input.training_model_type=='majority'):
# #     # pdb.set_trace()
# #     model = MajorityLabelVoter()
# # else:
# model = LabelModel(cardinality=2, verbose=True, device='cpu')
# model.fit(L_train=initial_vectors, n_epochs=500, log_freq=100, seed=123)
# # snorkel needs to get an estimator using fit function first
# # training model with all labelling functions

# probs_test= model.predict_proba(L=initial_vectors)

/tmp/ipykernel_30718/881225267.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sentences_df=pd.read_sql(f'SELECT * FROM amazon01', conn)


In [27]:
sentences_df.shape

(19943, 5)

In [31]:
sentences_df=sentences_df.head(1000)

In [32]:
corpus = sentences_df['text'].to_list()

In [35]:
model = SentenceTransformer('all-MiniLM-L6-v2')

# user_sentence_index = 0
# user_sentence = corpus[user_sentence_index]
corpus_embeddings = model.encode(corpus)



INFO 2024-05-03 18:53:16,258 [SentenceTransformer.py:__init__:66] Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO 2024-05-03 18:53:16,489 [SentenceTransformer.py:__init__:105] Use pytorch device: cpu


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [37]:
user_sentence = 'five stars'
sentence_embedding = model.encode(user_sentence)
similarities = util.pytorch_cos_sim(sentence_embedding, corpus_embeddings)[0]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [36]:

top_results = np.argsort(-similarities)[0:10] 

print(f"Top 10 similar sentences to: '{user_sentence}'")
for idx in top_results:
    print(corpus[idx], "(Score: {:.4f})\n".format(similarities[idx]))
    

Top 10 similar sentences to: 'five stars'
five stars great (Score: 0.9146)

five stars great (Score: 0.9146)

five stars great (Score: 0.9146)

five stars good (Score: 0.9102)

five stars good (Score: 0.9102)

five stars good (Score: 0.9102)

five stars good (Score: 0.9102)

five stars good (Score: 0.9102)

five stars ok (Score: 0.9047)

five stars as expected (Score: 0.8826)

